#Import Tweets Data 


1.   Import Tweets from sntwitter API (https://github.com/JustAnotherArchivist/snscrape)

2.   Clean up data - Remove special characters, emojis, memes in each tweets

In [2]:
pip install snscrape

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
import snscrape

In [4]:
import numpy as np
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import matplotlib.pyplot as mlpt
import csv
#pip install snscrape
!pip install snscrape
import datetime as dt
import snscrape.modules.twitter as sntwitter

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple


In [47]:
number = []
extreme_positive = []
positive = []
neutral = []
negative = []
extreme_negative = []

In [6]:
import unicodedata
import contractions
from contractions import contractions_dict
import re
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import spacy
nlp = spacy.load('en_core_web_sm')
import nltk
#nltk.download('stopwords')
from nltk.tokenize.toktok import ToktokTokenizer
from autocorrect import spell

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text, contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        return expanded_contraction
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def stemmer(text):
    # Lancaster Stemmer
    ls = LancasterStemmer()
    #snowball stemmer
    #ss = SnowballStemmer()
    text = ' '.join([ls.stem(word) for word in text.split()])
    return text

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
def remove_stopwords(text, stopwords=stopword_list):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in stopwords]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def spell_check(text):
    spelled_text = [spell(w) for w in (nltk.word_tokenize(text))]
    return spelled_text

In [7]:
def normalize_string(s):
    
    #Lower case
    #s = s.lower()
    #remove HTML
    s = re.sub('<[^<]+?>','', s)
    #remove numbers
    s = ''.join(c for c in s if not c.isdigit())
    #expand contractions 
    #s = expand_contractions(remove_accented_chars(s),contractions_dict)
    ##s = remove_accented_chars(s)
    # remove extra newlines
    s = re.sub(r'[\r|\n|\r\n]+', ' ',s)
    #remove special characters
    s = remove_special_characters(s)
    s = re.sub('_', ' ', s)
    # remove extra whitespace
    s = re.sub(' +', ' ', s)
    #Remove stopwords
    s = remove_stopwords(s)
    #Lemmatize
    #s = lemmatize_text(s)
    #spell check
    #s = spell_check(s)
    #print(1)
    return s

In [8]:
tweets = []
# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#rhino  since:2021-01-01 until:2021-10-10','corona lang:en').get_items()):
    tweets.append([tweet.date,  tweet.content])

In [9]:
tweets = pd.DataFrame(tweets, columns=['Date', 'Tweets'])

In [10]:
data=tweets
cdata=pd.DataFrame(columns=['Date','Tweets'])
index=0
for index,row in data.iterrows():
    stre=row["Tweets"]
    my_new_string = re.sub('[^ a-zA-Z0-9]', '', stre)
    cdata.sort_index()
    cdata.at[index,'Date']=row["Date"]
    cdata.at[index,'Tweets']=my_new_string
    index=index+1
tweets=cdata
tweets['Date'] = tweets['Date'].astype('datetime64[ns]')
tweets["Date"]=tweets["Date"].dt.date

In [11]:
tweets

,Date,Tweets
0,2021-10-09,New post httpstcoiTXpkxzT31 art cartoon Digi...
1,2021-10-09,Finished making RhinoDragon20 as a Nintendo Mi...
2,2021-10-09,Happy birthday to the man beast rhyno themanbe...
3,2021-10-09,Electing the wrong people to power and creatin...
4,2021-10-09,Great job by Rhino ReinhardtCole FutureSen Bel...
...,...,...
11097,2021-01-01,Good Morning from Chitwan National Park wildli...
11098,2021-01-01,These things are a little different chiapparh...
11099,2021-01-01,Introducing the Elasmotheriumelasmotherium mam...
11100,2021-01-01,Every time we sit down for dinner if you look ...


In [12]:
tweets["Tweets"] =tweets["Tweets"].apply(lambda x: normalize_string(x))

In [13]:
tweets["Tweets"] = tweets["Tweets"].apply(lambda x: ' '.join(word for word in x.split(' ') if not word.startswith('http')))

In [14]:
tweets

,Date,Tweets
0,2021-10-09,New post art cartoon DigitalIllustration eanim...
1,2021-10-09,Finished making RhinoDragon Nintendo MiiIts ba...
2,2021-10-09,Happy birthday man beast rhyno themanbeastrhyn...
3,2021-10-09,Electing wrong people power creating kleptocra...
4,2021-10-09,Great job Rhino ReinhardtCole FutureSen Bellev...
...,...,...
11097,2021-01-01,Good Morning Chitwan National Park wildlife ch...
11098,2021-01-01,These things little different chiapparhino rhi...
11099,2021-01-01,Introducing Elasmotheriumelasmotherium mammal ...
11100,2021-01-01,Every time sit dinner look see frenchbulldogs ...


In [15]:
tweets.to_csv('2021-1031.csv')

In [16]:
tweets = []
# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#rhino  since:2019-01-01 until:2019-12-31','corona lang:en').get_items()):
    tweets.append([tweet.date,  tweet.content])
    
tweets = pd.DataFrame(tweets, columns=['Date', 'Tweets'])
data=tweets
cdata=pd.DataFrame(columns=['Date','Tweets'])
index=0
for index,row in data.iterrows():
    stre=row["Tweets"]
    my_new_string = re.sub('[^ a-zA-Z0-9]', '', stre)
    cdata.sort_index()
    cdata.at[index,'Date']=row["Date"]
    cdata.at[index,'Tweets']=my_new_string
    index=index+1
tweets=cdata
tweets['Date'] = tweets['Date'].astype('datetime64[ns]')
tweets["Date"]=tweets["Date"].dt.date

tweets["Tweets"] =tweets["Tweets"].apply(lambda x: normalize_string(x))
tweets["Tweets"] = tweets["Tweets"].apply(lambda x: ' '.join(word for word in x.split(' ') if not word.startswith('http')))

tweets.to_csv('2019-1231.csv')

In [164]:
tweets = pd.read_csv('2019-1031.csv').dropna()

In [166]:
import pandas as pd
import os
 
 #读取news_data.csv，保存到新建的news_data.txt中
with open('rhino_2021.txt','a+', encoding='utf-8') as f:
    for line in tweets.values:
        if 'RhinoGirls' not in str(line[2]) and 'amp' not in str(line[2]) and 'Mulch' not in str(line[2]):
            f.write((str(line[2])+'\n'))

#Sentiment Analysis for Tweets

Sentiment Analysis is a process of ‘computationally’ evaluating whether a piece of text is positive, negative or neutral. In our case, conducting sentiment analysis on tweets help determine the public's moods towards BLM(Black Lives Matter) topic.
1.   **VADER (Valence Aware Dictionary and Entiment Reasoner)**

```
"VADER is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed
in social media. A sentiment lexicon is a list of lexical features (e.g., words) which are generally labeled
according to their semantic orientation aseither positive or negative. VADER not only tells about the Positivity
and Negativity score but also tells us about how positive or negative a sentiment is." (GreeksforGreeks)
```
2.   **Reference**
 
  Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

In [1]:
!pip install vaderSentiment
!pip install pyspark

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple


In [ ]:
import pyspark as spark
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
import re
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [37]:
tweets = pd.read_csv('2014-1231.csv')
tweets = tweets.dropna().reset_index()

In [33]:
tweets1['type']  = tweets1['Tweets'].apply(lambda x: 0 if type(x)==str else 1)

In [36]:
tweets1[tweets1['type']==1]

,Unnamed: 0,Date,Tweets,type
34686,34686,2014-06-24,NaN,1


In [43]:
def sentiment_calculation(data):
    #tweets = pd.read_csv(file)
    FullDataTw=sql.createDataFrame(data)
    FullDataTw = FullDataTw.dropna()
    FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")
 #setting column names of Twitter dataset
    CleanDF = FullDataTw.withColumnRenamed('Tweets', 'Tweet')
    CleanDF = FullDataTw.withColumnRenamed('Date', 'Date_Time')
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    analyser = SentimentIntensityAnalyzer()
    def senti_score_udf(sentence):
        snt = analyser.polarity_scores(sentence)
        return ([snt['neg'], snt['neu'], snt['pos'], snt['compound']])
    func_udf2 = udf(senti_score_udf, ArrayType(FloatType()))
    CleanDF = CleanDF.withColumn('p_neg', func_udf2(CleanDF['Tweets'])[0])
    CleanDF = CleanDF.withColumn('p_neu', func_udf2(CleanDF['Tweets'])[1])
    CleanDF = CleanDF.withColumn('p_pos', func_udf2(CleanDF['Tweets'])[2])
    CleanDF = CleanDF.withColumn('p_comp', func_udf2(CleanDF['Tweets'])[3])
    #CleanDF.show(120)
    CleanDF.toPandas().to_csv('sen.csv')
    
    tw=pd.read_csv("sen.csv")
    tw=tw.drop(columns=["Unnamed: 0"])
    tw["length"]=""
    i=0
    # iterate through the csv file 
    for val in tw["Tweets"]: 
      val = str(val) 
      tokens = val.split() 
      tw["length"][i]=len(tokens)
      i=i+1
    tw=tw.drop(columns=["p_neg","p_neu","p_pos"])

    tw["p_comp"].describe()
    mean=tw["p_comp"].mean()
    std=tw["p_comp"].std()
    tw["sentiment"]=""
    i=0
    for val in tw["p_comp"]: 
      if val> mean+2*std:
        tw["sentiment"][i]="extremely positive"
      if val< mean-2*std:
        tw["sentiment"][i]="extremely negative"
      if val>mean+std and val<mean+2*std:
        tw["sentiment"][i]="positive"
      if val<mean-std and val>mean-2*std:
        tw["sentiment"][i]="negative"
      if val>mean-std and val<mean+std:
        tw["sentiment"][i]="neutral"
      i=i+1

    sentiment=tw.groupby(tw['sentiment']).count().reset_index()
    return sentiment

In [109]:
tweets = pd.read_csv('2021-1031.csv')
tweets = tweets.dropna().reset_index()

In [110]:
sentiment = sentiment_calculation(tweets)
sentiment

,sentiment,index,Unnamed: 0.1,Date_Time,Tweets,p_comp,length
0,extremely negative,319,319,319,319,319,319
1,negative,1262,1262,1262,1262,1262,1262
2,neutral,7212,7212,7212,7212,7212,7212
3,positive,2308,2308,2308,2308,2308,2308


In [111]:
len(tweets)

11101

In [112]:
number.append(len(tweets))
extreme_positive.append(0)
positive.append(sentiment['Tweets'][3])
neutral.append(sentiment['Tweets'][2])
negative.append(sentiment['Tweets'][1])
extreme_negative.append(sentiment['Tweets'][0])

In [92]:
number.append(len(tweets))
extreme_positive.append(sentiment['Tweets'][1])
positive.append(sentiment['Tweets'][4])
neutral.append(sentiment['Tweets'][3])
negative.append(sentiment['Tweets'][2])
extreme_negative.append(sentiment['Tweets'][0])

In [114]:
year = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]

In [115]:
tweet_sentiment={"Year" : year,"number" : number, "extreme_positive": extreme_positive, "positive": positive,
                  "neutral": neutral, "negative": negative, "extreme_negative" : extreme_negative}
tweet_sentiment=pd.DataFrame(tweet_sentiment)

In [117]:
tweet_sentiment['sentiment score'] = tweet_sentiment['extreme_positive']*2+tweet_sentiment['positive']-tweet_sentiment['extreme_negative']*2-tweet_sentiment['negative']

In [119]:
tweet_sentiment.to_csv('sentiment.csv')

In [10]:
tweets = pd.read_csv('2014-1231.csv')